In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dense, Dropout

# Fetch stock data
def fetch_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        print("No data fetched. Please check the ticker or date range.")
        return None
    return data

# Prepare dataset
def create_dataset(data, time_step):
    X, Y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)




# Prepare dataset
def create_dataset(data, time_step):
    X, Y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# **Bayesian Neural Network (BNN)**
class BayesianNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.3):
        super(BayesianNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def train_bnn(X_train, y_train, X_test):
    model = BayesianNN(input_dim=X_train.shape[1], hidden_dim=64, output_dim=1)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    X_test = torch.tensor(X_test, dtype=torch.float32)

    # Training loop
    epochs = 500
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

    # Predictions
    model.eval()
    with torch.no_grad():
        predictions = model(X_test).numpy()

    return predictions, model

# **Feedforward Neural Network**
class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.2):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def train_feedforward(X_train, y_train, X_test):
    model = FeedforwardNN(input_dim=X_train.shape[1], hidden_dim=64, output_dim=1)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)

    # Training loop
    epochs = 500
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output.squeeze(), y_train)
        loss.backward()
        optimizer.step()

    # Predictions
    model.eval()
    with torch.no_grad():
        predictions = model(X_test).numpy()

    return predictions, model

# **LSTM**
def train_lstm(X_train, y_train, X_test):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
    predictions = model.predict(X_test)

    return predictions, model

# **GRU**
def train_gru(X_train, y_train, X_test):
    model = Sequential()
    model.add(GRU(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(GRU(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
    predictions = model.predict(X_test)

    return predictions, model

# **Bidirectional LSTM**
def train_bidirectional_lstm(X_train, y_train, X_test):
    model = Sequential()
    model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(50, return_sequences=False)))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
    predictions = model.predict(X_test)

    return predictions, model

# Predict future prices
def predict_future_prices(model, scaler, data, time_step, future_days, model_type='LSTM'):
    last_data = data[-time_step:]
    last_data = last_data.reshape(1, time_step, 1)
    
    future_predictions = []
    
    for _ in range(future_days):
        if model_type in ['LSTM', 'GRU', 'BidirectionalLSTM']:
            future_pred = model.predict(last_data)
        else:
            future_pred = model(torch.tensor(last_data, dtype=torch.float32).view(1, -1)).detach().numpy()

        future_predictions.append(future_pred[0, 0])
        last_data = np.append(last_data[:, 1:, :], future_pred.reshape(1, 1, 1), axis=1)

    # Inverse transform the predicted prices
    future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

    return future_predictions

#-----------------Function to generate various plots---------------------------------------------------------------------
def plot_stock_price(data):
    plt.plot(data['Close'], color='blue', label='Stock Price')
    plt.title('Stock Price Over Time')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

def plot_moving_averages(data):
    data['50_MA'] = data['Close'].rolling(window=50).mean()
    data['200_MA'] = data['Close'].rolling(window=200).mean()
    plt.plot(data['Close'], color='blue', label='Stock Price')
    plt.plot(data['50_MA'], color='orange', label='50-Day MA')
    plt.plot(data['200_MA'], color='green', label='200-Day MA')
    plt.title('Stock Price and Moving Averages')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

def plot_candlestick(data):
    import plotly.graph_objects as go
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])
    fig.update_layout(title='Candlestick Chart', xaxis_title='Date', yaxis_title='Price')
    fig.show()

def plot_volume(data):
    plt.bar(data.index, data['Volume'], color='gray')
    plt.title('Stock Trading Volume')
    plt.xlabel('Date')
    plt.ylabel('Volume')
    plt.show()

def plot_rsi(data):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    plt.plot(data.index, rsi, label='RSI', color='purple')
    plt.title('Relative Strength Index (RSI)')
    plt.xlabel('Date')
    plt.ylabel('RSI')
    plt.axhline(70, color='red', linestyle='--')
    plt.axhline(30, color='green', linestyle='--')
    plt.legend()
    plt.show()

def plot_macd(data):
    short_ema = data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = data['Close'].ewm(span=26, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=9, adjust=False).mean()

    plt.plot(data.index, macd, label='MACD', color='blue')
    plt.plot(data.index, signal, label='Signal Line', color='red')
    plt.title('MACD (Moving Average Convergence Divergence)')
    plt.xlabel('Date')
    plt.ylabel('MACD Value')
    plt.legend()
    plt.show()

def plot_bollinger_bands(data):
    moving_avg = data['Close'].rolling(window=20).mean()
    rolling_std = data['Close'].rolling(window=20).std()
    
    upper_band = moving_avg + (rolling_std * 2)
    lower_band = moving_avg - (rolling_std * 2)

    plt.plot(data['Close'], label='Stock Price')
    plt.plot(moving_avg, label='20-Day Moving Average', color='orange')
    plt.plot(upper_band, label='Upper Band', color='red')
    plt.plot(lower_band, label='Lower Band', color='green')
    plt.title('Bollinger Bands')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

def plot_all_analysis(data):
    plot_stock_price(data)
    plot_moving_averages(data)
    plot_candlestick(data)
    plot_volume(data)
    plot_rsi(data)
    plot_macd(data)
    plot_bollinger_bands(data)

# User interface to choose between plot analysis and model prediction
def choose_analysis_or_model():
    def handle_plot_analysis():
        print("\nChoose the plot for analysis:")
        print("1. Stock Price Line Plot (Time Series Plot)")
        print("2. Moving Averages (50-Day, 200-Day)")
        print("3. Candlestick Chart")
        print("4. Volume Plot")
        print("5. Relative Strength Index (RSI)")
        print("6. MACD (Moving Average Convergence Divergence)")
        print("7. Bollinger Bands")
        print("8. All of the Above")
        
        try:
            plot_choice = int(input("Enter the number corresponding to your choice: "))
            ticker = input("Enter the stock ticker (e.g., AAPL): ")
            start_date = input("Enter the start date (YYYY-MM-DD): ")
            end_date = input("Enter the end date (YYYY-MM-DD): ")

            data = fetch_data(ticker, start_date, end_date)
            if data is None:
                return
            
            # Call corresponding plot function
            if plot_choice == 1:
                plot_stock_price(data)
            elif plot_choice == 2:
                plot_moving_averages(data)
            elif plot_choice == 3:
                plot_candlestick(data)
            elif plot_choice == 4:
                plot_volume(data)
            elif plot_choice == 5:
                plot_rsi(data)
            elif plot_choice == 6:
                plot_macd(data)
            elif plot_choice == 7:
                plot_bollinger_bands(data)
            elif plot_choice == 8:
                plot_all_analysis(data)
            else:
                print("Invalid choice. Please select a valid option.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    def handle_model_prediction():
        print("\nChoose the model for prediction:")
        print("1. Bayesian Neural Network (BNN)")
        print("2. LSTM")
        print("3. GRU")
        print("4. Feedforward Neural Network (FFNN)")
        print("5. Bayesian LSTM")
        print("6. Bidirectional LSTM")
        
        try:
            model_choice = int(input("Enter the number corresponding to your choice: "))
            ticker = input("Enter the stock ticker (e.g., AAPL): ")
            start_date = input("Enter the start date (YYYY-MM-DD): ")
            end_date = input("Enter the end date (YYYY-MM-DD): ")
            time_step = int(input("Enter the time step (e.g., 50): "))
            future_days = int(input("Enter the number of future days to predict: "))
            
            # Fetch and preprocess data
            data = fetch_data(ticker, start_date, end_date)
            if data is None:
                return

            close_prices = data['Close'].values.reshape(-1, 1)
            scaler = MinMaxScaler(feature_range=(0, 1))
            scaled_data = scaler.fit_transform(close_prices)
            
            X, y = create_dataset(scaled_data, time_step)
            X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape for LSTM/GRU models
            
            # Train and predict with selected model
            if model_choice == 1:  # Bayesian Neural Network
                predictions, model = train_bnn(X.reshape(X.shape[0], -1), y, X.reshape(X.shape[0], -1))
                future_predictions = predict_next_days_bnn(model, scaler, scaled_data, time_step, future_days)
            elif model_choice == 2:  # LSTM
                predictions, model = train_lstm(X, y, X)
                future_predictions = predict_next_days_lstm_gru(model, scaler, scaled_data, time_step, future_days, "LSTM")
            elif model_choice == 3:  # GRU
                predictions, model = train_gru(X, y, X)
                future_predictions = predict_next_days_lstm_gru(model, scaler, scaled_data, time_step, future_days, "GRU")
            elif model_choice == 4:  # Feedforward Neural Network
                predictions, model = train_feedforward(X.reshape(X.shape[0], -1), y, X.reshape(X.shape[0], -1))
                future_predictions = predict_next_days_ffnn(model, scaler, scaled_data, time_step, future_days)
            elif model_choice == 5:  # Bayesian LSTM
                predictions, model = train_bayesian_lstm(X, y, X)
                future_predictions = predict_next_days_lstm_gru(model, scaler, scaled_data, time_step, future_days, "LSTM")
            elif model_choice == 6:  # Bidirectional LSTM
                predictions, model = train_bidirectional_lstm(X, y, X)
                future_predictions = predict_next_days_lstm_gru(model, scaler, scaled_data, time_step, future_days, "LSTM")
            else:
                print("Invalid choice. Please select a valid option.")
                return

            # Plot actual vs predicted and future predictions
            plt.figure(figsize=(10, 6))
            plt.plot(data.index[-len(y):], scaler.inverse_transform(y.reshape(-1, 1)), label="Actual Prices")
            plt.plot(data.index[-len(predictions):], scaler.inverse_transform(predictions), label="Predicted Prices")
            plt.title(f"{ticker} Price Prediction")
            plt.legend()
            plt.show()

            future_dates = pd.date_range(start=data.index[-1], periods=future_days + 1, freq='B')[1:]
            plt.plot(data['Close'], label='Actual Price')
            plt.plot(future_dates, future_predictions, color='green', label='Future Predictions')
            plt.title(f"{ticker} Future Price Predictions ({future_days} days)")
            plt.legend()
            plt.show()
        except ValueError:
            print("Invalid input. Please enter valid numerical values.")

    print("Choose the analysis type:")
    print("1. Analysis using Plots")
    print("2. Prediction using Models")

    try:
        choice = int(input("Enter the number corresponding to your choice: "))
        if choice == 1:
            handle_plot_analysis()
        elif choice == 2:
            handle_model_prediction()
        else:
            print("Invalid choice. Please select 1 or 2.")
    except ValueError:
        print("Invalid input. Please enter a number.")


In [9]:
choose_analysis_or_model()

Choose the analysis type:
1. Analysis using Plots
2. Prediction using Models


Enter the number corresponding to your choice:  2



Choose the model for prediction:
1. Bayesian Neural Network (BNN)
2. LSTM
3. GRU
4. Feedforward Neural Network (FFNN)
5. Bayesian LSTM
6. Bidirectional LSTM


Enter the number corresponding to your choice:  1
Enter the stock ticker (e.g., AAPL):  aapl
Enter the start date (YYYY-MM-DD):  2020-02-02
Enter the end date (YYYY-MM-DD):  2024-02-02
Enter the time step (e.g., 50):  50
Enter the number of future days to predict:  50


[*********************100%***********************]  1 of 1 completed


NameError: name 'predict_next_days_bnn' is not defined